# 核心组件: 消息总线 (Message Bus)

`消息总线 (MessageBus)` 是多代理系统的中央通信骨干。它实现了**发布-订阅 (publish-subscribe)** 设计模式，允许代理之间在不直接相互感知的情况下进行通信。这种解耦是构建复杂、可扩展和可维护的基于代理的系统的关键特性。

### 工作原理
1. **主题 (Topics)**: 消息按主题进行组织，主题是简单的字符串 (例如, `"state.reservoir.level"`)。
2. **发布 (Publishing)**: 一个代理可以向特定`主题`发送一条`消息` (一个Python字典)。
3. **订阅 (Subscribing)**: 一个代理可以向特定`主题`注册一个`监听器`函数 (一个回调函数)。 

当一条消息被发布到一个主题时，消息总线会立即调用所有订阅了该确切主题的监听器函数，并将消息传递给它们。

**注意:** 这是一个简单的实现，不支持通配符订阅等高级功能。

## 演示

在下面的例子中，我们将创建一个消息总线和几个监听器。我们会将它们订阅到不同的主题，然后发布一系列消息。输出将显示哪个监听器收到了哪条消息，从而演示了基于主题的路由机制。

In [ ]:
from swp.central_coordination.collaboration.message_bus import MessageBus, Message

# 1. 创建消息总线
bus = MessageBus()

# 2. 定义一些监听器函数
def reservoir_level_listener(message: Message):
    print(f"[水库监听器] 收到: {message}")

def gate_listener(message: Message):
    print(f"[闸门监听器] 收到: {message}")
    
def all_commands_listener(message: Message):
    print(f"[命令监听器] 收到: {message}")

# 3. 将监听器订阅到主题
print("--- 订阅监听器 ---")
bus.subscribe("state.reservoir.level", reservoir_level_listener)
bus.subscribe("state.gate.opening", gate_listener)
bus.subscribe("command.gate.setpoint", all_commands_listener)
bus.subscribe("command.pump.status", all_commands_listener)
print("----------------------------\n")

# 4. 发布消息
print("--- 发布消息 ---")
print("发布一个水库水位...")
bus.publish("state.reservoir.level", {'value': 15.2, 'units': 'm'})

print("\n发布一个闸门命令...")
bus.publish("command.gate.setpoint", {'value': 0.8, 'units': 'percent'})

print("\n发布一个闸门状态 (无活跃监听器)...")
bus.publish("state.gate.opening", {'value': 0.75})

print("\n发布一个水泵命令...")
bus.publish("command.pump.status", {'value': 1, 'note': 'turn on'})
print("---------------------------")

### 输出分析

正如输出所示：
- `reservoir_level_listener` 只收到了发布到 `state.reservoir.level` 的消息。
- `gate_listener` 只收到了 `state.gate.opening` 的消息。
- `all_commands_listener` 收到了发布到 `command.gate.setpoint` 和 `command.pump.status` 的两条消息，因为我们分别将它订阅到了这两个主题。
- 对于 `state.gate.opening` 消息，没有监听器被调用，因为 `gate_listener` 是在消息发布*之后*才订阅的。这表明订阅不是追溯性的。